In [1]:
import os
import gc
from dataclasses import dataclass
from typing import Optional, Dict, Any

import torch
import torch.nn as nn
import torch.nn.functional as F

from transformers import AutoConfig, AutoTokenizer, LlamaModel
from peft import LoraConfig, get_peft_model, TaskType


c:\Users\korea\miniconda3\envs\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
from transformers import AutoTokenizer, AutoModel, AutoConfig

model_id = "meta-llama/Llama-3.1-8B" 

hf_config = AutoConfig.from_pretrained(model_id)

# hidden state 뽑기 좋은 AutoModel 사용
backbone = AutoModel.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",)

        # 1) backbone freeze
for p in backbone.parameters():
    p.requires_grad = False

tokenizer = AutoTokenizer.from_pretrained(model_id)

backbone.eval()

tokenizer.pad_token = tokenizer.eos_token
# padding='max_length', max_length = 4800

print(backbone)

Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.36s/it]
Some parameters are on the meta device because they were offloaded to the cpu.


LlamaModel(
  (embed_tokens): Embedding(128256, 4096)
  (layers): ModuleList(
    (0-31): 32 x LlamaDecoderLayer(
      (self_attn): LlamaAttention(
        (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
        (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
        (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
        (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
      )
      (mlp): LlamaMLP(
        (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
        (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
        (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
        (act_fn): SiLU()
      )
      (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
    )
  )
  (norm): LlamaRMSNorm((4096,), eps=1e-05)
  (rotary_emb): LlamaRotaryEmbedding()
)


In [5]:
@dataclass
class DownstreamConfig:
    model_id: str = "meta-llama/Meta-Llama-3.1-8B"  # base로 쓰는 걸 추천(인스트럭트도 가능)
    num_labels: int = 2
    task: str = "classification"  # "classification" or "regression"
    pooling: str = "last"         # "last" or "mean"
    dora_r: int = 8
    dora_alpha: int = 16
    dora_dropout: float = 0.0
    target_modules: tuple = ("q_proj", "v_proj")  # attention에만
    torch_dtype: torch.dtype = torch.bfloat16

args = DownstreamConfig()

In [ ]:
TaskType.

<enum 'TaskType'>

In [ ]:
# PEFT 설정
peft_args = LoraConfig(
    task_type=TaskType.FEATURE_EXTRACTION,
    r=args.dora_r,
    lora_alpha=args.dora_alpha,
    lora_dropout=args.dora_dropout,
    target_modules=list(args.target_modules),
    bias="none",
    use_dora=True,)